In [1]:
from google.colab import drive
drive.mount('/content/drive')
PROJECT = '/content/drive/MyDrive/guvi_hcl_project3'
print("PROJECT path is:", PROJECT)


Mounted at /content/drive
PROJECT path is: /content/drive/MyDrive/guvi_hcl_project3


In [2]:
!mkdir -p "{PROJECT}/data/raw" "{PROJECT}/data/processed" "{PROJECT}/notebooks" "{PROJECT}/src" "{PROJECT}/figures" "{PROJECT}/reports"
!ls -la "{PROJECT}"


total 20
drwx------ 4 root root 4096 Aug 30 04:40 data
drwx------ 2 root root 4096 Aug 30 04:40 figures
drwx------ 2 root root 4096 Aug 30 04:40 notebooks
drwx------ 2 root root 4096 Aug 30 04:40 reports
drwx------ 2 root root 4096 Aug 30 04:40 src


In [3]:
!pip install --quiet pandas numpy matplotlib plotly scikit-learn openpyxl


In [4]:
import pandas as pd, numpy as np
np.random.seed(42)
n = 8000
start = pd.to_datetime('2024-09-01')
order_dates = start + pd.to_timedelta(np.random.randint(0, 365, size=n), unit='D')
ship_offsets = np.random.randint(0,3,size=n)
delivery_offsets = np.random.randint(1,11,size=n)
ship_dates = order_dates + pd.to_timedelta(ship_offsets, unit='D')
delivery_dates = ship_dates + pd.to_timedelta(delivery_offsets, unit='D')

categories = ['Clothing','Electronics','Home & Kitchen','Beauty','Sports','Toys']
states = ['Karnataka','Maharashtra','Tamil Nadu','Delhi','Uttar Pradesh','West Bengal']
sellers = [f'seller_{i}' for i in range(1,51)]
payment = ['COD','UPI','Card','Wallet']

df = pd.DataFrame({
    'order_item_id': range(1,n+1),
    'order_id': np.random.randint(100000,200000,size=n),
    'order_date': order_dates,
    'ship_date': ship_dates,
    'delivery_date': delivery_dates,
    'customer_id': np.random.randint(10000,20000,size=n),
    'city': np.random.choice(['Bengaluru','Mumbai','Chennai','New Delhi','Lucknow','Kolkata'], n),
    'state': np.random.choice(states, n),
    'product_id': np.random.randint(1000,2000,size=n),
    'category': np.random.choice(categories, n, p=[0.25,0.2,0.2,0.15,0.1,0.1]),
    'seller_id': np.random.choice(sellers, n),
    'quantity': np.random.randint(1,4,size=n),
    'unit_price': np.round(np.random.uniform(100,5000,size=n),2),
    'discount': np.round(np.random.uniform(0,300,size=n),2),
    'payment_method': np.random.choice(payment, n)
})
prob_return = np.where(df['category']=='Clothing', 0.12, 0.03)
df['is_returned'] = (np.random.rand(n) < prob_return).astype(int)
df['return_reason'] = df['is_returned'].apply(lambda x: np.random.choice(['Size issue','Damaged','Not as described','Other']) if x==1 else '')
df['sla_days'] = 5
outpath = f"{PROJECT}/data/raw/ecom_sample_orders.csv"
df.to_csv(outpath, index=False)
print("Saved synthetic dataset to:", outpath)
df.head()


Saved synthetic dataset to: /content/drive/MyDrive/guvi_hcl_project3/data/raw/ecom_sample_orders.csv


order_item_id  order_id order_date  ship_date delivery_date  customer_id  \
0              1    128964 2024-12-12 2024-12-13    2024-12-23        11067   
1              2    197065 2025-08-15 2025-08-15    2025-08-22        15252   
2              3    176992 2025-05-29 2025-05-29    2025-06-01        15863   
3              4    199259 2024-12-16 2024-12-16    2024-12-19        10379   
4              5    142240 2024-11-11 2024-11-13    2024-11-15        19476   

        city          state  product_id        category  seller_id  quantity  \
0  New Delhi  Uttar Pradesh        1903  Home & Kitchen  seller_21         2   
1     Mumbai      Karnataka        1469  Home & Kitchen  seller_43         1   
2  New Delhi    Maharashtra        1360  Home & Kitchen   seller_6         1   
3    Lucknow      Karnataka        1853  Home & Kitchen  seller_25         1   
4     Mumbai     Tamil Nadu        1594          Sports  seller_41         1   

   unit_price  discount payment_method  is_returned     return_reason  \
0     4893.73     60.74         Wallet            0                     
1     2519.51    140.55            UPI            1  Not as described   
2     3040.14    149.68           Card            0                     
3     1227.50    279.37         Wallet            0                     
4     1764.34    160.85            COD            0                     

   sla_days  
0         5  
1         5  
2         5  
3         5  
4         5

In [9]:
import pandas as pd, numpy as np
pd.options.display.max_columns = 60

INPUT = f"{PROJECT}/data/raw/ecom_sample_orders.csv"
OUT = f"{PROJECT}/data/processed/fact_orders.csv"

df = pd.read_csv(INPUT, parse_dates=['order_date','ship_date','delivery_date'])
print("Raw rows:", len(df))

df = df.drop_duplicates()
for c in ['category','payment_method','city','state','seller_id','return_reason']:
    if c in df.columns:
        df[c] = df[c].astype(str).str.strip()

df = df[~df['order_date'].isnull()]
df['ship_date'] = df['ship_date'].fillna(df['order_date'] + pd.to_timedelta(1, unit='D'))
df['delivery_date'] = df['delivery_date'].fillna(df['ship_date'] + pd.to_timedelta(3, unit='D'))


df['delivery_days'] = (df['delivery_date'] - df['ship_date']).dt.days.clip(lower=0)
if 'sla_days' in df.columns:
    df['sla_days'] = pd.to_numeric(df['sla_days'], errors='coerce').fillna(5).astype(int)
else:
    df['sla_days'] = 5
df['on_time'] = (df['delivery_days'] <= df['sla_days']).astype(int)


if 'is_returned' not in df.columns:
    if 'return_status' in df.columns:
        df['is_returned'] = df['return_status'].map({'returned':1}).fillna(0).astype(int)
    else:
        df['is_returned'] = df['return_reason'].notnull().astype(int) if 'return_reason' in df.columns else 0


df['quantity'] = pd.to_numeric(df.get('quantity', 1), errors='coerce').fillna(1).astype(int)
df['unit_price'] = pd.to_numeric(df.get('unit_price', 0), errors='coerce').fillna(0.0)
df['discount'] = pd.to_numeric(df.get('discount', 0), errors='coerce').fillna(0.0)


df['gmv'] = df['quantity'] * (df['unit_price'] - df['discount'])


order_agg = df.groupby(['order_id','category','state','city','seller_id']).agg(
    order_date=('order_date','min'),
    customer_id=('customer_id','first'),
    total_items=('order_item_id','count'),
    order_gmv=('gmv','sum'),
    any_returned=('is_returned','max'),
    avg_delivery_days=('delivery_days','mean'),
    on_time_pct=('on_time','mean')
).reset_index()

order_agg['is_returned'] = order_agg['any_returned'].astype(int)
order_agg['on_time'] = (order_agg['on_time_pct'] >= 1.0).astype(int)


order_agg.to_csv(OUT, index=False)
print("✅ Saved processed orders to:", OUT)
print("Final columns:", order_agg.columns.tolist())
order_agg.head(8)


Raw rows: 8000
✅ Saved processed orders to: /content/drive/MyDrive/guvi_hcl_project3/data/processed/fact_orders.csv
Final columns: ['order_id', 'category', 'state', 'city', 'seller_id', 'order_date', 'customer_id', 'total_items', 'order_gmv', 'any_returned', 'avg_delivery_days', 'on_time_pct', 'is_returned', 'on_time']


order_id        category          state       city  seller_id order_date  \
0    100016  Home & Kitchen     Tamil Nadu  Bengaluru  seller_47 2025-08-15   
1    100036            Toys     Tamil Nadu  Bengaluru   seller_6 2025-05-27   
2    100046        Clothing          Delhi     Mumbai   seller_9 2025-01-18   
3    100051  Home & Kitchen      Karnataka  New Delhi  seller_21 2024-09-01   
4    100056  Home & Kitchen     Tamil Nadu    Kolkata  seller_15 2025-07-12   
5    100063     Electronics  Uttar Pradesh    Lucknow  seller_38 2025-05-11   
6    100100          Beauty  Uttar Pradesh    Kolkata  seller_38 2024-10-22   
7    100125  Home & Kitchen     Tamil Nadu    Kolkata  seller_28 2025-01-04   

   customer_id  total_items  order_gmv  any_returned  avg_delivery_days  \
0        16883            1    5283.74             0                4.0   
1        11144            1     963.60             0                1.0   
2        14627            1    1588.90             0                2.0   
3        14643            1    5732.72             0                3.0   
4        13410            1    5443.82             0                2.0   
5        12390            1    2889.70             0                1.0   
6        13950            1    4191.18             1                1.0   
7        16473            1    5723.46             0                2.0   

   on_time_pct  is_returned  on_time  
0          1.0            0        1  
1          1.0            0        1  
2          1.0            0        1  
3          1.0            0        1  
4          1.0            0        1  
5          1.0            0        1  
6          1.0            1        1  
7          1.0            0        1

In [10]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

orders = pd.read_csv(f"{PROJECT}/data/processed/fact_orders.csv", parse_dates=['order_date'])
print("Total processed order rows:", len(orders))

total_orders = orders['order_id'].nunique()
return_rate = orders['is_returned'].mean()
on_time_pct = orders['on_time'].mean()
gmv = orders['order_gmv'].sum()
aov = orders['order_gmv'].mean()

print(f"Total orders: {total_orders}")
print(f"Return rate: {return_rate:.2%}")
print(f"On-time %: {on_time_pct:.2%}")
print(f"GMV: {gmv:.2f}")
print(f"AOV: {aov:.2f}")

# weekly trend (orders and returns)
orders.set_index('order_date', inplace=True)
weekly = orders.resample('W').agg({'order_id':'nunique','is_returned':'sum','order_gmv':'sum'}).rename(columns={'order_id':'orders','is_returned':'returns'})
fig, ax = plt.subplots(figsize=(12,4))
ax.plot(weekly.index, weekly['orders'], label='Orders')
ax.plot(weekly.index, weekly['returns'], label='Returns')
ax.set_title('Weekly orders & returns')
ax.legend()
plt.tight_layout()
fig_path = f"{PROJECT}/figures/weekly_orders_returns.png"
plt.savefig(fig_path, bbox_inches='tight')
print("Saved plot to:", fig_path)
plt.show()


Total processed order rows: 8000
Total orders: 7703
Return rate: 5.53%
On-time %: 50.08%
GMV: 38661200.77
AOV: 4832.65
Saved plot to: /content/drive/MyDrive/guvi_hcl_project3/figures/weekly_orders_returns.png


In [11]:
!ls -lah "{PROJECT}/data" "{PROJECT}/data/raw" "{PROJECT}/data/processed" "{PROJECT}/figures"
import pandas as pd
pd.read_csv(f"{PROJECT}/data/processed/fact_orders.csv").head(10)


/content/drive/MyDrive/guvi_hcl_project3/data:
total 8.0K
drwx------ 2 root root 4.0K Aug 30 05:52 processed
drwx------ 2 root root 4.0K Aug 30 04:41 raw

/content/drive/MyDrive/guvi_hcl_project3/data/processed:
total 709K
-rw------- 1 root root 709K Aug 30 05:52 fact_orders.csv

/content/drive/MyDrive/guvi_hcl_project3/data/raw:
total 952K
-rw------- 1 root root 952K Aug 30 04:41 ecom_sample_orders.csv

/content/drive/MyDrive/guvi_hcl_project3/figures:
total 115K
-rw------- 1 root root 24K Aug 30 04:52 category_return_rate.png
-rw------- 1 root root 24K Aug 30 04:58 state_delivery_days.png
-rw------- 1 root root 20K Aug 30 04:55 top_sellers_return_rate.png
-rw------- 1 root root 48K Aug 30 05:53 weekly_orders_returns.png


order_id        category          state       city  seller_id  order_date  \
0    100016  Home & Kitchen     Tamil Nadu  Bengaluru  seller_47  2025-08-15   
1    100036            Toys     Tamil Nadu  Bengaluru   seller_6  2025-05-27   
2    100046        Clothing          Delhi     Mumbai   seller_9  2025-01-18   
3    100051  Home & Kitchen      Karnataka  New Delhi  seller_21  2024-09-01   
4    100056  Home & Kitchen     Tamil Nadu    Kolkata  seller_15  2025-07-12   
5    100063     Electronics  Uttar Pradesh    Lucknow  seller_38  2025-05-11   
6    100100          Beauty  Uttar Pradesh    Kolkata  seller_38  2024-10-22   
7    100125  Home & Kitchen     Tamil Nadu    Kolkata  seller_28  2025-01-04   
8    100166     Electronics    West Bengal    Kolkata  seller_10  2025-01-05   
9    100168  Home & Kitchen  Uttar Pradesh  Bengaluru  seller_47  2025-05-10   

   customer_id  total_items  order_gmv  any_returned  avg_delivery_days  \
0        16883            1    5283.74             0                4.0   
1        11144            1     963.60             0                1.0   
2        14627            1    1588.90             0                2.0   
3        14643            1    5732.72             0                3.0   
4        13410            1    5443.82             0                2.0   
5        12390            1    2889.70             0                1.0   
6        13950            1    4191.18             1                1.0   
7        16473            1    5723.46             0                2.0   
8        18008            1    1345.51             0                7.0   
9        14418            1    7020.40             0                5.0   

   on_time_pct  is_returned  on_time  
0          1.0            0        1  
1          1.0            0        1  
2          1.0            0        1  
3          1.0            0        1  
4          1.0            0        1  
5          1.0            0        1  
6          1.0            1        1  
7          1.0            0        1  
8          0.0            0        0  
9          1.0            0        1

In [12]:
readme = f"""
# E-Commerce Orders & Returns Analytics

Project root: {PROJECT}

## Objective
Identify drivers of returns and late deliveries and recommend actions to reduce return rate and improve on-time delivery.

## Folders
- data/raw: raw CSV(s)
- data/processed: cleaned, aggregated data
- notebooks: notebooks for prep, EDA, viz
- figures: images/plots
- reports: final PDF & slides

## Quick run
1. Open notebooks/01_prep_ecom.ipynb in Colab (this notebook).
2. Run cells in order: mount → generate/upload → prep → EDA.

"""
with open(f"{PROJECT}/README.md", "w") as f:
    f.write(readme)
print("README created at", f"{PROJECT}/README.md")


README created at /content/drive/MyDrive/guvi_hcl_project3/README.md
